In [33]:
# evaluation of the best logistic regression 
# best recall C =  0.8

In [34]:
import pandas as pd
# plot the number of unique trees vs sentence length 
from pandas.api.types import CategoricalDtype
from plotnine import *
from plotnine.data import mpg
%matplotlib inline

import math
from collections import Counter
import numpy as np
import pickle
from nltk import Tree
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression

In [35]:
feature = pd.read_csv('final_data_cleaned.csv')

# drop the zero infection rate 
feature_c = feature[feature['INFECTION RATE'] != 0]
feature_p = feature[feature['INFECTION RATE'] == 0]

X_p = feature_p.iloc[:,4:].drop('INFECTION RATE',axis=1).to_numpy()
X = feature_c.iloc[:,4:].drop('INFECTION RATE',axis=1).to_numpy() # no population 
y_raw = np.array(feature_c['INFECTION RATE'])

mid = np.median(y_raw)
y = [1 if y_raw[i] >= mid else 0 for i in range(len(y_raw)) ]

In [36]:
# train on all the data obtained on 04/12/2020
# predict and evaluate on the up-to-data data 05/17/2020
transformer = Normalizer(norm = 'l1').fit(X) # norm = ‘l1’, ‘l2’,'max'
X_norm = transformer.transform(X) 
X_p_norm = transformer.transform(X_p)


clf = LogisticRegression(random_state=0,solver ='liblinear',max_iter=100, C = 0.8)# solver = 'sag', 'saga' or 'liblinear'
clf.fit(X_norm, y)
y_pred = clf.predict(X_p_norm)

pred = pd.DataFrame(list(feature_p['COUNTY']),columns = ["COUNTY"])
pred['STATE'] = list(feature_p['STATE'])
pred['pred'] = y_pred
pred[pred['pred']==1]

,COUNTY,STATE,pred
141,Floyd County,Kentucky,1
148,Knox County,Kentucky,1
158,Rowan County,Kentucky,1
169,Chippewa County,Michigan,1
207,Laclede County,Missouri,1
208,Lawrence County,Missouri,1
210,Marion County,Missouri,1
314,Bronx County,New York,1
315,Kings County,New York,1
316,Queens County,New York,1


In [37]:
# evaluate on the on up-to-date data 
# load data 
update = pd.read_csv('final_data_update.csv')
svm_result = pd.read_csv('219.csv')
ls_result = pred[pred['pred']==1]

In [38]:
update_rate = update['5/17/20'] / update['POPULATION']
update['infection_rate'] = update_rate
mid = np.median(update_rate)
print(mid) 

0.0009560800717060054


In [39]:
# get missing data 
middle_data = pd.merge(feature_p,update, on='COUNTY', how='inner')
print(middle_data.shape)
missing_data = middle_data[middle_data['STATE_x'] == middle_data['STATE_y']]
print(missing_data.shape)

(1937, 107)
(529, 107)


In [40]:
high_risk_reg = missing_data[missing_data['infection_rate']>=mid]
high_risk_reg.shape

(67, 107)

In [41]:
# get recall + 4 new york counties
middle_data = pd.merge(svm_result,high_risk_reg, on='COUNTY', how='inner')
print(middle_data.shape)
final_merge = middle_data[middle_data['STATE_x'] == middle_data['STATE_y']]
print(final_merge.shape)

(28, 109)
(28, 109)


In [42]:
# get recall + 4 new york counties
middle_data = pd.merge(ls_result,high_risk_reg, on='COUNTY', how='inner')
print(middle_data.shape)
final_merge = middle_data[middle_data['STATE_x'] == middle_data['STATE_y']]
print(final_merge.shape)

(2, 109)
(2, 109)
